In [1]:
import config
import pandas as pd
import psycopg2
import numpy as np

- Anesthésie-réanimation (code 3) : La consultation pré-anesthésique
- Code acte de consultation : CS_+MPC+MCS
- Base de 25 ou 30 euros (à voir)


In [59]:
study_id = 2
sql = f"select * from depassement_study where id={study_id}"
df_study = pd.read_sql(sql, config.connection_string)
df_study

,id,profession_type,datesource_min,datesource_max,tarif_s1
0,2,Anest,2301,2308,30.0


In [60]:
profession_type = df_study.iloc[0]["profession_type"]
datesource_min = df_study.iloc[0]["datesource_min"]
datesource_max = df_study.iloc[0]["datesource_max"]
tarif_s1 = df_study.iloc[0]["tarif_s1"]
profession_type, datesource_min, datesource_max, tarif_s1

('Anest', 2301, 2308, 30.0)

In [4]:
sql = f"""
select p.*, t.*, tds.date_source_id, b.id as adresse_id, an.cp as cp, ar.dept_id as dept_id, b.code_insee from ps p
join tarif t on t.ps_id = p.id
join tarif_date_source tds on tds.tarif_id = t.id
join cabinet c on t.cabinet_id = c.id
join adresse_raw ar on c.adresse_raw_id = ar.id
join adresse_norm an on ar.adresse_norm_id = an.id
join ban b on an.ban_id = b.id
--join ps_cabinet_date_source pcd on pcd.ps_id = p.id and pcd.cabinet_id = c.id and pcd.date_source_id = tds.date_source_id
join profession_type pt on pt.profession = '{profession_type}' and t.profession_id = pt.profession_id
where tds.date_source_id >= {datesource_min} and  tds.date_source_id <= {datesource_max}
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

df = pd.read_csv("../data/depassement/anest.csv", low_memory=False)
# df = pd.read_sql(sql, config.connection_string)
df  #1235287 





,id,genre,key,nom,prenom,has_inpp,rule_nb,id.1,profession_id,mode_exercice_id,nature_id,convention_id,option_contrat,vitale,code,ps_id,cabinet_id,famille_acte_id,montant,borne_inf,borne_sup,montant_2,borne_inf_2,borne_sup_2,montant_imagerie,borne_inf_imagerie,borne_sup_imagerie,montant_anesthesie,borne_inf_anesthesie,borne_sup_anesthesie,montant_cec,borne_inf_cec,borne_sup_cec,tarif_id,date_source_id,id.2,nom.1,key.1,telephone,adresse_raw_id,id.3,adresse2,adresse3,adresse4,cp,commune,adresse_norm_id,dept_id,id.4,numero,rue1,rue2,cp.1,commune.1,osm_id,osm_score,ban_id,ban_score,source_id,lon,lat,score,dept_id.1,iris,id.5,adresse_id,numero.1,rep,nom_voie,code_postal,code_insee,nom_commune,nom_ancienne_commune,lon.1,lat.1,libelle_acheminement,nom_afnor,dept_id.2,is_lieu_dit
0,701635,H,810003092342,LEGRAND,ERIC,True,1.0,21597107,3,NaN,3,3,False,True,AHPA0090,701635,202743,82.0,182.6,172.6,182.6,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21597107,2301,202743,LEGRAND ERIC,LEGRAND_ERIC_01280_180_ROUTE_DU_NANT,NaN,267,267,CLINIQUE CENDANEG,180 ROUTE DU NANT,NaN,1280,PREVESSIN MOENS,260,1,260,180.0,ROUTE DU NANT,CLINIQUE CENDANEG,1280,PREVESSIN MOENS,NaN,NaN,83627,1.000000,2,6.086309,46.248438,1.000000,1,013130000,83627,01313_0074_00180,180.0,NaN,ROUTE DU NANT,1280,1313,PREVESSIN MOENS,NaN,6.086309,46.248438,NaN,NaN,1,False
1,648294,H,810100116853,MEYNIEL,PIERRE,True,1.0,18759498,3,NaN,3,3,False,True,HHFE0060,648294,443,112.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18759498,2301,443,MEYNIEL PIERRE,MEYNIEL_PIERRE_01000_62_AVENUE_DE_JASSERON,474456495.0,21,21,CLINIQUE CONVERT,62 AVENUE DE JASSERON,NaN,1000,BOURG EN BRESSE,21,1,21,62.0,AVENUE DE JASSERON,CLINIQUE CONVERT,1000,BOURG EN BRESSE,NaN,NaN,227083,1.000000,2,5.253583,46.211282,1.000000,1,010530502,227083,01053_1950_00062,62.0,NaN,AVENUE DE JASSERON,1000,1053,BOURG EN BRESSE,NaN,5.253583,46.211282,NaN,NaN,1,False
2,648325,H,810100159663,JACQUIOT,NICOLAS,True,1.0,21599300,3,NaN,3,3,False,True,LMMC0010,648325,147,54.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21599300,2301,147,JACQUIOT NICOLAS,JACQUIOT_NICOLAS_01000_62_AVENUE_DE_JASSERON,474456495.0,128,128,NaN,62 AVENUE DE JASSERON,NaN,1000,BOURG EN BRESSE,126,1,126,62.0,AVENUE DE JASSERON,NaN,1000,BOURG EN BRESSE,NaN,NaN,227083,1.000000,3,5.253603,46.211526,1.000000,1,010530502,227083,01053_1950_00062,62.0,NaN,AVENUE DE JASSERON,1000,1053,BOURG EN BRESSE,NaN,5.253583,46.211282,NaN,NaN,1,False
3,757555,H,810003100152,BIOT,LOIC PIERRE,True,7.0,21599440,3,NaN,3,3,False,True,JGFA0090,757555,180,28.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21599440,2301,180,CLINIQUE CONVERT,CLINIQUE_CONVERT_01000_ROUTE_DE_JASSERON,474456495.0,186,186,URGENCES,ROUTE DE JASSERON,NaN,1000,BOURG EN BRESSE,183,1,183,NaN,ROUTE DE JASSERON,URGENCES,1000,BOURG EN BRESSE,NaN,NaN,227037,0.939286,2,5.242986,46.210987,0.939286,1,010530502,227037,01053_1950_00002,2.0,NaN,AVENUE DE JASSERON,1000,1053,BOURG EN BRESSE,NaN,5.242986,46.210987,NaN,NaN,1,False
4,648329,H,810100179646,COLLIN,ANTHONY,True,1.0,15847529,3,NaN,3,3,False,True,JGFA0060,648329,148,28.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15847529,2301,148,COLLIN ANTHONY,COLLIN_ANTHONY_01000_62_AVENUE_DE_JASSERON,474456495.0,21,21,CLINIQUE CONVERT,62 AVENUE DE JASSERON,NaN,1000,BOURG EN BRESSE,21,1,21,62.0,AVENUE DE JASSERON,CLINIQUE CONVERT,1000,BOURG EN BRESSE,NaN,NaN,227083,1.000000,2,5.253583,46.211282,1.000000,1,010530502,227083,01053_1950_00062,62.0,NaN,AVENUE DE JASSERON,1000,1053,BOURG EN BRESSE,NaN,5.253583,46.211282,NaN,NaN,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235282,750139,H,810100609014,KHELIFA,ISMAIL,True,1.0,23455359,3,NaN,3,3,T

In [5]:
# Renommer les colonnes
df.rename(columns={
    'genre': 'gender',
    'prenom': 'prénom',
    'code': 'codeccamdelacte',
    'nature_id': 'naturedexercice',
    'convention_id': 'convention',
    'code_insee': 'codeinsee',
    'cp': 'matchcp',
    'montant': 'montantgénéralementconstaté',
    'borne_inf': 'borneinférieuredumontant',
    'borne_sup': 'bornesupérieuredumontant',
    'option_contrat': 'optioncontratdaccèsauxsoins',
    'dept_id': 'dep',
}, inplace=True)

# Colonnes à conserver
df = df[['ps_id', 'gender', 'nom', 'prénom', 'codeccamdelacte', 'naturedexercice', 'convention',
         'codeinsee', 'montantgénéralementconstaté', 'borneinférieuredumontant', 'bornesupérieuredumontant',
         'adresse_id', 'date_source_id', 'optioncontratdaccèsauxsoins', 'matchcp', 'dep']]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep
0,701635,H,LEGRAND,ERIC,AHPA0090,3,3,1313,182.6,172.6,182.6,01313_0074_00180,2301,False,1280,1
1,648294,H,MEYNIEL,PIERRE,HHFE0060,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1
2,648325,H,JACQUIOT,NICOLAS,LMMC0010,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1
3,757555,H,BIOT,LOIC PIERRE,JGFA0090,3,3,1053,0.0,0.0,0.0,01053_1950_00002,2301,False,1000,1
4,648329,H,COLLIN,ANTHONY,JGFA0060,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235282,750139,H,KHELIFA,ISMAIL,JKFE0020,3,3,94052,0.0,92.8,242.8,94052_9915_00003,2308,True,94130,94
1235283,737272,H,GAFTARNIK,MICHEL,JHFA0130,3,3,59606,0.0,0.0,0.0,59606_3870_00010,2308,True,59300,59
1235284,799528,H,COSTILLE,PAUL,BBCA0030,3,3,69389,0.0,0.0,0.0,69389_0860_00480,2308,True,69009,69
1235285,797974,H,ASSELBORN,THEO,LMMA0040,3,3,69266,0.0,0.0,0.0,69266_2350_00158,2308,True,69100,69


In [6]:
# Clé combinée
pd.options.mode.copy_on_write = True
df['b'] = df['ps_id'].astype(str) + "_" + df['adresse_id'].astype(str)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
0,701635,H,LEGRAND,ERIC,AHPA0090,3,3,1313,182.6,172.6,182.6,01313_0074_00180,2301,False,1280,1,701635_01313_0074_00180
1,648294,H,MEYNIEL,PIERRE,HHFE0060,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1,648294_01053_1950_00062
2,648325,H,JACQUIOT,NICOLAS,LMMC0010,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1,648325_01053_1950_00062
3,757555,H,BIOT,LOIC PIERRE,JGFA0090,3,3,1053,0.0,0.0,0.0,01053_1950_00002,2301,False,1000,1,757555_01053_1950_00002
4,648329,H,COLLIN,ANTHONY,JGFA0060,3,3,1053,0.0,0.0,0.0,01053_1950_00062,2301,False,1000,1,648329_01053_1950_00062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235282,750139,H,KHELIFA,ISMAIL,JKFE0020,3,3,94052,0.0,92.8,242.8,94052_9915_00003,2308,True,94130,94,750139_94052_9915_00003
1235283,737272,H,GAFTARNIK,MICHEL,JHFA0130,3,3,59606,0.0,0.0,0.0,59606_3870_00010,2308,True,59300,59,737272_59606_3870_00010
1235284,799528,H,COSTILLE,PAUL,BBCA0030,3,3,69389,0.0,0.0,0.0,69389_0860_00480,2308,True,69009,69,799528_69389_0860_00480
1235285,797974,H,ASSELBORN,THEO,LMMA0040,3,3,69266,0.0,0.0,0.0,69266_2350_00158,2308,True,69100,69,797974_69266_2350_00158


In [7]:
nb_ps_id = df["ps_id"].nunique()
nb_ps_id  #3384 


3384

In [8]:
nb_b = df["b"].nunique()
nb_b  #4017

4017

In [9]:
df = df.sort_values(by='ps_id')
df['unique'] = df.groupby('ps_id')['adresse_id'].transform('nunique')
df['un'] = df['unique']
pd.set_option('display.max_rows', 200)
df.head(10)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un
266071,647964,F,BENYOUB DA SILVA,KARIMA,NFKA0070,5,3,1053,218.7,0.0,0.0,01053_3125_00006,2306,True,1000,1,647964_01053_3125_00006,1,1
926559,647964,F,BENYOUB DA SILVA,KARIMA,LMMC0020,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2303,True,1000,1,647964_01053_3125_00006,1,1
143423,647964,F,BENYOUB DA SILVA,KARIMA,HMFC0040,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2308,True,1000,1,647964_01053_3125_00006,1,1
197082,647964,F,BENYOUB DA SILVA,KARIMA,QZMA0010,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2305,True,1000,1,647964_01053_3125_00006,1,1
195856,647964,F,BENYOUB DA SILVA,KARIMA,QZMA0010,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2304,True,1000,1,647964_01053_3125_00006,1,1
925251,647964,F,BENYOUB DA SILVA,KARIMA,LMMC0020,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2302,True,1000,1,647964_01053_3125_00006,1,1
264850,647964,F,BENYOUB DA SILVA,KARIMA,NFKA0070,5,3,1053,218.7,0.0,0.0,01053_3125_00006,2305,True,1000,1,647964_01053_3125_00006,1,1
194626,647964,F,BENYOUB DA SILVA,KARIMA,QZMA0010,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2303,True,1000,1,647964_01053_3125_00006,1,1
296736,647964,F,BENYOUB DA SILVA,KARIMA,NEKA0200,5,3,1053,229.9,229.9,476.0,01053_3125_00006,2307,True,1000,1,647964_01053_3125_00006,1,1
711480,647964,F,BENYOUB DA SILVA,KARIMA,EDAF0030,5,3,1053,0.0,0.0,0.0,01053_3125_00006,2308,True,1000,1,647964_01053_3125_00006,1,1


In [10]:
# Comptages globaux
df['NB_total'] = df.groupby('dep')['ps_id'].transform('nunique')
df = df.sort_values(by='dep')
df["NB_total"].unique()

array([ 24,  13,  19,   3,   8, 105,  20,   7,  11,  16,   4, 196,  44,
         5,  18,  26,   9,  30,   2,  27,  14,  10,   6,  34, 138, 134,
       101,  63,  33,  69,  38, 112,  32,  15,  60,  54, 117,  22,  62,
        35,  77,  37,  48, 185,  28,  25,  42, 186,  17,  71,  45,  52,
        98,  47,  53,  39], dtype=int64)

In [11]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2_total'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2_total'] = df.groupby('dep')['NB2_total'].transform('max')
df["NB2_total"] = df["NB2_total"].fillna(0)
df["NB2_total"].unique()



array([ 22.,   7.,  11.,   2.,   3.,  80.,  10.,   5.,   8.,   1.,  30.,
        15.,   4.,  28.,   0.,  21.,  13.,   6.,  12.,  49., 117.,  71.,
        27.,  26.,  42.,  33.,  73.,  48.,  31.,  44.,  17., 102.,  14.,
        57.,  45.,  29., 179.,  19.,  37., 153.,  61.,   9.,  52.,  35.,
        34.,  89.,  24.])

In [12]:
df['c'] = 1
df['NB_Ftotal'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_Ftotal"].unique() # 3384

array([3384], dtype=int64)

In [13]:
df['NB2_Ftotal'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_Ftotal'] = df['NB2_Ftotal'].max() 
df["NB2_Ftotal"].unique() # 2333

array([2333.])

In [14]:
# ////Algo pour corriger l'excès d'appareillage (à copier/coller pour APL?)////
# ///prenons les ind qui ont plusieurs type d'options///
df['cc'] = np.where(df['optioncontratdaccèsauxsoins'] == False, -1, np.where(df['optioncontratdaccèsauxsoins'], 1, np.nan))
df["cc"].unique()

array([ 1., -1.])

In [15]:
df['ccc'] = df.groupby('ps_id')['cc'].transform('mean')
df["ccc"].unique()

array([ 1.        , -1.        ,  0.75      , -0.96551724,  0.5       ,
        0.        ,  0.6       ,  0.66666667,  0.74572127, -0.8974359 ,
        0.71428571,  0.95238095,  0.92307692,  0.74696707, -0.99307958,
       -0.93548387, -0.94594595, -0.95789474,  0.65991903,  0.68      ,
       -0.95238095, -0.71428571, -0.75      , -0.33333333,  0.2195122 ,
       -0.03125   , -0.5       ,  0.25      ,  0.33333333, -0.008     ])

In [16]:
# . drop if ccc==-1 |ccc==1 (1,218,938 observations deleted) Reste 16349
# Unique = 57
nb_ccc_1 = len(df[(df["ccc"] == 1) | (df["ccc"] == -1)]) #1218938
nb_not_1 = len(df) - nb_ccc_1 # 16349
nb = len(df[(df["ccc"] != 1) & (df["ccc"] != -1)]["ps_id"].unique()) # 57
nb_ccc_1, nb_not_1, nb

(1218938, 16349, 57)

In [17]:
# Correction spécifique pour JERNITE
df.loc[(df['nom'] == "JERNITE") & (df['optioncontratdaccèsauxsoins'] == False), 'gender'] = "F"
df.loc[(df['nom'] == "JERNITE") & (df['optioncontratdaccèsauxsoins'] == False), 'ps_id'] += 1000000
df.loc[(df['nom'] == "JERNITE") & (df['optioncontratdaccèsauxsoins']), 'prénom'] = "MOHAMED"
df[df['nom'] == "JERNITE"]

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc


In [18]:
# Pondération
df['weight'] = df['un'].map({1: 1, 2: 0.5, 3: 0.33, 4: 0.25, 5: 0.2}).fillna(0)

In [19]:
# Tous les praticiens font des consultations
nb_ps_id, df["ps_id"].nunique()

(3384, 3384)

In [20]:
# Filtrage sur les actes de consultation
# On se concentre sur la consultation la plus standard: enfant de moins de 2 ans///
# Tarif S1 ou S2 OPTAM: 37 euros (23 (CS) +4 (MEP) +10 (NFP)) donc hors OPTAM/S1 on est à 33 euros
# code retenu: "CS_+MEP+NFP" et "CS_+NFP"
df = df[df['codeccamdelacte'].str.contains("CS", na=False)]
# 24911
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight
817096,753937,H,LONJARET,LAURENT,CS_,3,3,1053,50.0,23.0,50.0,01053_1950_00062,2308,False,1000,1,753937_01053_1950_00062,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0
814751,753937,H,LONJARET,LAURENT,CS_,3,3,1053,50.0,23.0,50.0,01053_1950_00062,2306,False,1000,1,753937_01053_1950_00062,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0
815931,753937,H,LONJARET,LAURENT,CS_,3,3,1053,50.0,23.0,50.0,01053_1950_00062,2307,False,1000,1,753937_01053_1950_00062,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0
808796,753937,H,LONJARET,LAURENT,CS_,3,3,1053,50.0,23.0,50.0,01053_1950_00062,2301,False,1000,1,753937_01053_1950_00062,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0
813577,753937,H,LONJARET,LAURENT,CS_,3,3,1053,50.0,23.0,50.0,01053_1950_00062,2305,False,1000,1,753937_01053_1950_00062,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191111,732401,F,PAPADACCI,MADELEINE,CS_+MPC+MCS,3,1,2B033,0.0,0.0,0.0,2b033_a165_00327,2308,False,20200,202,732401_2b033_a165_00327,2,2,7,0.0,1,3384,2333.0,-1.0,-1.0,0.5
185413,732401,F,PAPADACCI,MADELEINE,CS_+MPC+MCS,3,1,2B033,0.0,0.0,0.0,2b033_a165_00327,2303,False,20200,202,732401_2b033_a165_00327,2,2,7,0.0,1,3384,2333.0,-1.0,-1.0,0.5
187724,732401,F,PAPADACCI,MADELEINE,CS_+MPC+MCS,3,1,2B033,0.0,0.0,0.0,2b033_a165_00327,2305,False,20200,202,732401_2b033_a165_00327,2,2,7,0.0,1,3384,2333.0,-1.0,-1.0,0.5
1220032,659440,H,DUFLANC,DOMINIQUE,CS_+MPC+MCS,3,1,2B033,0.0,0.0,0.0,2b033_1945_00005,2305,False,20200,202,659440_2b033_1945_00005,1,1,7,0.0,1,3384,2333.0,-1.0,-1.0,1.0


In [21]:
df["codeccamdelacte"].value_counts(normalize=True) # 29 9 60

codeccamdelacte
CS_+MPC+MCS    0.604311
CS_            0.297499
CS_+MPC        0.098190
Name: proportion, dtype: float64

In [22]:
df["ps_id"].nunique() #2691

2691

In [23]:
df = df.sort_values(by='convention')
df.groupby('convention')["ps_id"].nunique() #1023 1669

convention
1    1023
3    1669
Name: ps_id, dtype: int64

In [24]:
# Moyenne par groupe
df = df.sort_values(by=['b',"convention", "codeccamdelacte", "date_source_id"])
df['mp'] = df.groupby(['b', 'convention', 'codeccamdelacte'])['montantgénéralementconstaté'].transform('mean')
df["mp"].unique()
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
68430,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2302,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
69700,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2303,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
850975,781771,H,NOUNOU,FADI,CS_,3,3,3098,33.0,33.0,33.0,03098_0040_00008,2304,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
904057,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
905359,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2302,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
906630,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2303,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
754802,781771,H,NOUNOU,FADI,CS_+MPC+MCS,3,3,3098,40.0,40.0,40.0,03098_0040_00008,2304,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0


In [25]:
# 0
len(df[(df['codeccamdelacte'] == "CS_+MEP+NFP") & (df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False)])

0

In [26]:
df.loc[(df['codeccamdelacte'] == "CS_+MEP+NFP") & (df['convention'] == 1) & (df['optioncontratdaccèsauxsoins'] == False), 'mp'] = tarif_s1
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
68430,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2302,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
69700,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2303,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
850975,781771,H,NOUNOU,FADI,CS_,3,3,3098,33.0,33.0,33.0,03098_0040_00008,2304,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
904057,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
905359,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2302,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
906630,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2303,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0
754802,781771,H,NOUNOU,FADI,CS_+MPC+MCS,3,3,3098,40.0,40.0,40.0,03098_0040_00008,2304,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0


****on élimine les duplications ps-adresse convention acte puis on sélectionne le prix le plus élevé d'un acte de consultation (discard tarif opposable des secteurs 2)
***puis on élimine les duplications ps-adresse convention (car les actes ont le même prix maintenant pour chaque ps-adresse)

In [27]:
# Efface 21669
# Reste 3242
df = df.drop_duplicates(subset=['b', 'convention', 'codeccamdelacte'])
df
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0
904057,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0


In [28]:
df['prixmoyen'] = df.groupby(['b', 'convention'])['mp'].transform('max')
pd.set_option('display.max_rows', 100)
df.head(20)
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp,prixmoyen
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0,40.0
904057,781771,H,NOUNOU,FADI,CS_+MPC+MCS,2,3,3098,40.0,40.0,40.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,40.0,40.0


In [29]:
# Reste 2810
df = df.drop_duplicates(subset=['b', 'convention'])
df
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)
# df_backup = df.copy()

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp,prixmoyen
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0,40.0


In [30]:
df.groupby('convention')["ps_id"].nunique() #1023 1669

convention
1    1023
3    1669
Name: ps_id, dtype: int64

In [31]:
# df = df_backup.copy()
df = df.drop_duplicates(subset=['b']) # inutile
print(len(df[df["mp"] == 0])) #874
df['prixmoyen'] = df["prixmoyen"].replace(0, tarif_s1) # /!\ BUG ne pas faire df["mp"].replace(0, tarif_s1)
# # ////Prix OK ////
# # df
df[(df["dep"]==3) & (df["nom"]=="NOUNOU")].head(100)

874


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp,prixmoyen
67136,781771,H,NOUNOU,FADI,CS_,2,3,3098,33.0,33.0,33.0,03098_0040_00008,2301,False,3630,3,781771_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,-1.0,-1.0,1.0,33.0,40.0


In [32]:
# Calculs départementaux
df = df.sort_values(by="dep")
df['NB'] = df.groupby('dep')['ps_id'].transform('nunique')
df["NB"].unique()

array([ 17,  12,  19,   2,   6,  64,   5,  10,  13,   3, 102,  41,   4,
        18,  24,   7,  27,  16,  33,  20, 134,  89,  92,  56,  15,  65,
        32, 104,  29,  14,  59,  11,  52,  26,  99,  54,  34,  63,  47,
       156,  23, 111,  72,  21,  42,  31,  49,  22,   8,   9,  38,   1],
      dtype=int64)

In [33]:
df['NB_F'] = df.groupby('c')['ps_id'].transform('nunique')
df["NB_F"].unique() # 2691

array([2691], dtype=int64)

In [34]:
mask_s2 = (df['convention'].isin([2, 3])) | ((df['convention'] == 1) & (df['optioncontratdaccèsauxsoins']))
df['NB2'] = df[mask_s2].groupby('dep')['ps_id'].transform('nunique')
df['NB2'] = df.groupby("dep")["NB2"].transform("mean")
df["NB2"] = df["NB2"].fillna(0)
df["NB2"].unique()

array([ 17.,   6.,  11.,   1.,  49.,   9.,   2.,   5.,   0.,  36.,  27.,
        15.,  26.,   4.,   8.,  21.,   3.,  46.,  82.,  66.,  12.,  40.,
        71.,  25.,  48.,  31.,  44.,  88.,  45.,  28., 150.,  14.,  23.,
        93.,  59.,  33.,  10.,   7.,  54.,  30.,  20.])

In [35]:
df['NB2_F'] = df[mask_s2].groupby('c')['ps_id'].transform('nunique')
df['NB2_F'] = df['NB2_F'].mean()
df["NB2_F"].unique()  # 1831

array([1831.])

In [36]:
df['ShareS2'] = df['NB2'] / df['NB']
# df[df["dep"]==1]['NB'].unique(), df[df["dep"]==1]['NB2'].unique(), df[df["dep"]==1]['ShareS2'].unique()
df['ShareS2'].unique()

array([1.        , 0.5       , 0.57894737, 0.16666667, 0.765625  ,
       0.47368421, 0.4       , 0.46153846, 0.        , 0.35294118,
       0.65853659, 0.61111111, 0.625     , 0.71428571, 0.96296296,
       0.21052632, 0.77777778, 0.91666667, 0.6       , 0.33333333,
       0.34328358, 0.92134831, 0.7173913 , 0.375     , 0.25      ,
       0.8       , 0.61538462, 0.2       , 0.75      , 0.84375   ,
       0.68269231, 0.86206897, 0.21428571, 0.81355932, 0.09090909,
       0.96875   , 0.84615385, 0.66666667, 0.65384615, 0.27272727,
       0.88888889, 0.6875    , 0.81481481, 0.32352941, 0.93617021,
       0.875     , 0.96153846, 0.60869565, 0.91304348, 0.88461538,
       0.83783784, 0.81944444, 0.52380952, 0.85714286, 0.87096774,
       0.06666667, 0.52941176, 0.67346939, 0.70833333, 0.77272727,
       0.125     , 0.11111111, 0.60526316, 0.96428571, 0.78947368,
       0.64516129])

In [37]:
df['ShareS2_F'] = df['NB2_F'] / df['NB_F']
df['ShareS2_F'].unique() #0.68

array([0.6804162])

In [38]:
# Dépassement
df['exessr'] = ((df['prixmoyen'] - tarif_s1) / tarif_s1) * 100
df['exessr'].unique()

array([ 66.66666667,   0.        , 116.66666667,  33.33333333,
       100.        ,  50.        ,  10.        , -23.33333333,
        83.33333333, 143.33333333, 103.33333333, 103.66666667,
       166.66666667, 126.66666667,  70.        , -16.66666667,
        35.        ,  -3.33333333,  46.66666667,  21.66666667,
        36.66666667,  80.        ,  53.33333333,  73.33333333,
        26.66666667,  40.        ,  75.        , 106.66666667,
        46.25      ,  16.66666667, 183.33333333, 150.        ,
        56.66666667,  60.        , 133.33333333,  70.33333333,
        38.33333333,  23.33333333,  86.66666667,  43.33333333,
        65.        , 110.        ,  53.66666667,  60.33333333,
       200.        , 170.33333333, 120.33333333, 400.        ,
       233.33333333, 169.66666667,  87.        , 137.        ,
       136.66666667])

In [39]:
for seuil in [10, 25, 50, 75, 100]:
    df[f'c{seuil}'] = df.groupby('dep')['exessr'].transform(lambda x: (x >= seuil).sum())
    df[f'c{seuil}_F'] = (df['exessr'] >= seuil).sum()
    df[f'r{seuil}'] = (df[f'c{seuil}'] / df['NB']) * 100
    df[f'r{seuil}_F'] = df[f'c{seuil}_F'] / df['NB_F']
print("c100", df["c100"].unique())
print("c10", df["c10"].unique())
print("c25", df["c25"].unique())
print("c50", df["c50"].unique())
print("cX0_F", df["c10_F"].unique(), df["c25_F"].unique(), df["c50_F"].unique(), df["c100_F"].unique()) # 1465	1448	1164	333
print("rX0_F", df["r10_F"].unique(), df["r25_F"].unique(), df["r50_F"].unique(), df["r100_F"].unique())

c100 [ 0  1  5 17 12  2  3  8  6  4  9 21 35 68 10 27]
c10 [ 16   3   9   1  42   7   0   5  29  11  10  12   4  20   2  23  32  78
  64  43   8  25  48  38  31  41  14  74  37  39 129  17  88  36   6  27
  26  19  45  30]
c25 [ 16   3   9   1   0  42   7   5  29  11  10  12   4  19   2  23  32  78
  64   8  25  48  37  31  41  14  69  39 129  17  22  88  35   6  27  26
  45  30]
c50 [ 16   3   7   1   0  42   5  29   8   4  19  22   9  28  70  63   2  25
  21  23  31  39  10  37  17  34 120  12  13  14  88   6  27  26  45  30]
cX0_F [1465] [1448] [1164] [333]
rX0_F [0.54440728] [0.53808993] [0.43255295] [0.12374582]


****dépassement moyen par département (uniquement les acteurs  faisant du dépassement)

In [40]:
# Moyennes nationales
df['PF'] = df['prixmoyen'].mean() # 40.87
df["PF"].unique()

array([40.87013345])

In [41]:
df['PFS2'] = df[mask_s2]['prixmoyen'].mean()
df['PFS2'].unique()  # 45.77

array([45.77304809])

In [42]:
# Moyennes départementales
df['PrixMoyen'] = df.groupby('dep')['prixmoyen'].transform('mean')
df['PrixMoyen'].unique()

array([48.82352941, 35.76923077, 40.52631579, 45.        , 30.5       ,
       47.58769231, 35.25      , 30.        , 32.30769231, 36.5046729 ,
       34.87804878, 37.72222222, 35.52      , 31.66666667, 35.71428571,
       37.3452381 , 31.9375    , 47.03703704, 46.08333333, 41.66666667,
       48.        , 35.5       , 34.19402985, 48.40449438, 44.04557292,
       31.41071429, 41.88815789, 33.2       , 38.33333333, 38.        ,
       45.61764706, 36.40480769, 40.90697674, 36.85      , 43.10810811,
       33.33333333, 46.47592593, 42.        , 37.42307692, 31.81818182,
       34.        , 40.17788462, 46.4375    , 38.92727273, 35.02985075,
       33.42857143, 49.02040816, 47.25      , 47.90804598, 50.        ,
       40.76923077, 35.625     , 42.23043478, 45.24074074, 59.18288288,
       36.16438356, 36.95238095, 46.50714286, 44.        , 50.21212121,
       43.23529412, 37.51020408, 37.044     , 32.66666667, 37.72727273,
       40.5       , 31.11111111, 36.94736842, 56.        , 37.75

In [43]:
df['PrixMoyenS2'] = df[mask_s2].groupby('dep')['prixmoyen'].transform('mean')
df['PrixMoyenS2'].unique()

array([48.82352941, 40.71428571,         nan, 48.18181818, 60.        ,
       33.        , 52.864     , 41.66666667, 30.        , 35.        ,
       47.84615385, 37.40740741, 42.63636364, 38.625     , 40.        ,
       38.        , 37.52439024, 32.5       , 33.875     , 50.        ,
       47.54545455, 53.33333333, 43.75      , 42.2173913 , 49.97560976,
       49.2625    , 33.76190476, 48.07      , 46.        , 41.11111111,
       43.33333333, 48.31034483, 39.38169014, 42.02564103, 38.3877551 ,
       43.47222222, 49.34130435, 48.        , 41.35294118, 36.66666667,
       41.38172043, 53.90909091, 40.91111111, 37.17021277, 40.90909091,
       50.26086957, 49.71428571, 48.54761905, 46.47058824, 39.64285714,
       43.3952381 , 47.14583333, 64.8311828 , 37.5       , 43.27272727,
       49.25833333, 44.        , 53.82142857, 55.        , 41.15151515,
       40.35882353, 34.        , 42.        , 41.47826087, 56.9122807 ,
       45.5       , 59.67741935, 44.385     ])

In [44]:
# df['Prix'] = df.groupby('dep')['PrixMoyen'].transform('mean')
df['depmoyen'] = ((df['PrixMoyen'] - tarif_s1) / tarif_s1) * 100  # * 100.roound(5)
df["depmoyen"].unique()

array([62.74509804, 19.23076923, 35.0877193 , 50.        ,  1.66666667,
       58.62564103, 17.5       ,  0.        ,  7.69230769, 21.68224299,
       16.2601626 , 25.74074074, 18.4       ,  5.55555556, 19.04761905,
       24.48412698,  6.45833333, 56.79012346, 53.61111111, 38.88888889,
       60.        , 18.33333333, 13.9800995 , 61.34831461, 46.81857639,
        4.70238095, 39.62719298, 10.66666667, 27.77777778, 26.66666667,
       52.05882353, 21.34935897, 36.35658915, 22.83333333, 43.69369369,
       11.11111111, 54.91975309, 40.        , 24.74358974,  6.06060606,
       13.33333333, 33.92628205, 54.79166667, 29.75757576, 16.76616915,
       11.42857143, 63.40136054, 57.5       , 59.69348659, 66.66666667,
       35.8974359 , 18.75      , 40.76811594, 50.80246914, 97.27627628,
       20.54794521, 23.17460317, 55.02380952, 46.66666667, 67.37373737,
       44.11764706, 25.03401361, 23.48      ,  8.88888889, 25.75757576,
       35.        ,  3.7037037 , 23.15789474, 86.66666667, 25.83

In [45]:
df["depmoyen_F"] =((df['PF'] - tarif_s1) / tarif_s1) * 100
df["depmoyen_F"] = df["depmoyen_F"] #.round(3)
df["depmoyen_F"].unique() # 36.23

array([36.23377817])

In [46]:
df["PrixMoyenS2"] = df.groupby('dep')['PrixMoyenS2'].transform('mean')
df['depmoyenS2'] = ((df['PrixMoyenS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyenS2'] = df['depmoyenS2'] #.round(3)
df['depmoyenS2'].unique()

array([ 62.74509804,  35.71428571,  60.60606061, 100.        ,
        10.        ,  76.21333333,  38.88888889,   0.        ,
        16.66666667,          nan,  59.48717949,  24.69135802,
        42.12121212,  28.75      ,  33.33333333,  26.66666667,
        25.08130081,   8.33333333,  12.91666667,  66.66666667,
        58.48484848,  77.77777778,  45.83333333,  40.72463768,
        66.58536585,  64.20833333,  12.53968254,  60.23333333,
        53.33333333,  37.03703704,  44.44444444,  61.03448276,
        31.27230047,  40.08547009,  27.95918367,  44.90740741,
        64.47101449,  60.        ,  37.84313725,  22.22222222,
        37.9390681 ,  79.6969697 ,  36.37037037,  23.90070922,
        36.36363636,  67.53623188,  65.71428571,  61.82539683,
        54.90196078,  32.14285714,  44.65079365,  57.15277778,
       116.10394265,  25.        ,  44.24242424,  64.19444444,
        46.66666667,  79.4047619 ,  83.33333333,  37.17171717,
        34.52941176,  13.33333333,  40.        ,  38.26

In [47]:
df['depmoyen_FS2'] = ((df['PFS2'] - tarif_s1) / tarif_s1) * 100
df['depmoyen_FS2'].unique() # 52.57


array([52.57682696])

In [48]:
df = df.sort_values(by=['dep', 'depmoyen'], ascending=[True, False])
df = df.drop_duplicates(subset=['dep'])
df['depmoyen'] = df['depmoyen'].fillna(0)
pd.set_option('display.max_rows', 20)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,weight,mp,prixmoyen,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,exessr,c10,c10_F,r10,r10_F,c25,c25_F,r25,r25_F,c50,c50_F,r50,r50_F,c75,c75_F,r75,r75_F,c100,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
47278,648067,H,CLOSON,MATTHIEU,CS_,3,3,1053,50.0,50.0,50.0,01053_1950_00002,2301,False,1000,1,648067_01053_1950_00002,1,1,24,22.0,1,3384,2333.0,-1.0,-1.0,1.0,50.0,50.0,17,2691,17.0,1831.0,1.000000,0.680416,66.666667,16,1465,94.117647,0.544407,16,1448,94.117647,0.53809,16,1164,94.117647,0.432553,0,473,0.000000,0.175771,0,333,0.000000,0.123746,40.870133,45.773048,48.823529,48.823529,62.745098,36.233778,62.745098,52.576827
258648,735466,H,VAN HAECKE,BENOIT,CS_+MPC+MCS,3,1,2408,30.0,30.0,30.0,02408_2110_00113,2308,True,2000,2,735466_02408_2110_00113,2,2,13,7.0,1,3384,2333.0,1.0,1.0,0.5,30.0,30.0,12,2691,6.0,1831.0,0.500000,0.680416,0.000000,3,1465,25.000000,0.544407,3,1448,25.000000,0.53809,3,1164,25.000000,0.432553,1,473,8.333333,0.175771,1,333,8.333333,0.123746,40.870133,45.773048,35.769231,40.714286,19.230769,36.233778,35.714286,52.576827
1224638,649166,H,AITOUAL,ABDALLAH,CS_+MPC+MCS,4,1,3098,40.0,40.0,40.0,03098_0040_00008,2301,True,3630,3,649166_03098_0040_00008,1,1,19,11.0,1,3384,2333.0,1.0,1.0,1.0,40.0,40.0,19,2691,11.0,1831.0,0.578947,0.680416,33.333333,9,1465,47.368421,0.544407,9,1448,47.368421,0.53809,7,1164,36.842105,0.432553,5,473,26.315789,0.175771,5,333,26.315789,0.123746,40.870133,45.773048,40.526316,48.181818,35.087719,36.233778,60.606061,52.576827
424111,649376,F,FOUSSIER,ISABELLE,CS_+MPC+MCS,3,1,4112,0.0,0.0,0.0,04112_a079,2301,False,4100,4,649376_04112_a079,1,1,3,2.0,1,3384,2333.0,-1.0,-1.0,1.0,0.0,30.0,2,2691,1.0,1831.0,0.500000,0.680416,0.000000,1,1465,50.000000,0.544407,1,1448,50.000000,0.53809,1,1164,50.000000,0.432553,1,473,50.000000,0.175771,1,333,50.000000,0.123746,40.870133,45.773048,45.000000,60.000000,50.000000,36.233778,100.000000,52.576827
510189,649718,H,DEAU,EMMANUEL,CS_+MPC+MCS,3,1,5061,0.0,0.0,0.0,05061_0072_00002,2301,False,5000,5,649718_05061_0072_00002,1,1,8,3.0,1,3384,2333.0,-1.0,-1.0,1.0,0.0,30.0,6,2691,1.0,1831.0,0.166667,0.680416,0.000000,1,1465,16.666667,0.544407,0,1448,0.000000,0.53809,0,1164,0.000000,0.432553,0,473,0.000000,0.175771,0,333,0.000000,0.123746,40.870133,45.773048,30.500000,33.000000,1.666667,36.233778,10.000000,52.576827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665318,735274,H,FATHALLAH,BECHIR,CS_+MPC,4,1,93007,0.0,0.0,0.0,93007_1180_00007,2301,False,93150,93,735274_93007_1180_00007,1,1,47,33.0,1,3384,2333.0,-1.0,-1.0,1.0,0.0,30.0,16,2691,8.0,1831.0,0.500000,0.680416,0.000000,5,1465,31.250000,0.544407,5,1448,31.250000,0.53809,5,1164,31.250000,0.432553,2,473,12.500000,0.175771,2,333,12.500000,0.123746,40.870133,45.773048,37.750000,45.500000,25.833333,36.233778,51.666667,52.576827
87801,764721,H,MARCHAND,MAXIME,CS_,3,3,94018,60.0,60.0,60.0,94018_0620_00009,2301,False,94220,94,764721_94018_0620_00009,1,1,53,42.0,1,3384,2333.0,-1.0,-1.0,1.0,60.0,60.0,38,2691,30.0,1831.0,0.789474,0.680416,100.000000,30,1465,78.947368,0.544407,30,1448,78.947368,0.53809,30,1164,78.947368,0.432553,30,473,78.947368,0.175771,27,333,71.052632,0.123746,40.870133,45.773048,53.589744,59.677419,78.632479,36.233778,98.924731,52.576827
886325,746547,H,GIROMPAIRE,DIDIER,CS_+MPC+MCS,4,1,95476,0.0,0.0,0.0,95476_0233_00001,2301,False,95520,95,746547_95476_0233_00001,1,1,39,24.0,1,3384,2333.0,-1.0,-1.0,1.0,0.0,30.0,31,2691,20.0,1831.0,0.645161,0.680416,0.000000,16,1465,51.612903,0.54

In [49]:
# gender nom prénom naturedexercice convention optioncontratdaccèsauxsoins codeccamdelacte ps_id montantgénéralementconstaté borneinférieuredumontant bornesupérieuredumontant date_source_id adresse_id matchcp codeinsee b unique un weight mp prixmoyen exessr nb10 c10 nb25 c25 nb50 c50 nb75 c75 nb100 c100 exess
df = df.drop(["gender","nom","prénom","naturedexercice","convention","optioncontratdaccèsauxsoins","codeccamdelacte","ps_id","montantgénéralementconstaté","borneinférieuredumontant","bornesupérieuredumontant","date_source_id","adresse_id","matchcp","codeinsee","b","unique","un","weight","mp","prixmoyen","exessr","c10","c25","c50","c75","c100"], axis=1)
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2
47278,1,24,22.0,1,3384,2333.0,-1.0,-1.0,17,2691,17.0,1831.0,1.000000,0.680416,1465,94.117647,0.544407,1448,94.117647,0.53809,1164,94.117647,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,48.823529,48.823529,62.745098,36.233778,62.745098,52.576827
258648,2,13,7.0,1,3384,2333.0,1.0,1.0,12,2691,6.0,1831.0,0.500000,0.680416,1465,25.000000,0.544407,1448,25.000000,0.53809,1164,25.000000,0.432553,473,8.333333,0.175771,333,8.333333,0.123746,40.870133,45.773048,35.769231,40.714286,19.230769,36.233778,35.714286,52.576827
1224638,3,19,11.0,1,3384,2333.0,1.0,1.0,19,2691,11.0,1831.0,0.578947,0.680416,1465,47.368421,0.544407,1448,47.368421,0.53809,1164,36.842105,0.432553,473,26.315789,0.175771,333,26.315789,0.123746,40.870133,45.773048,40.526316,48.181818,35.087719,36.233778,60.606061,52.576827
424111,4,3,2.0,1,3384,2333.0,-1.0,-1.0,2,2691,1.0,1831.0,0.500000,0.680416,1465,50.000000,0.544407,1448,50.000000,0.53809,1164,50.000000,0.432553,473,50.000000,0.175771,333,50.000000,0.123746,40.870133,45.773048,45.000000,60.000000,50.000000,36.233778,100.000000,52.576827
510189,5,8,3.0,1,3384,2333.0,-1.0,-1.0,6,2691,1.0,1831.0,0.166667,0.680416,1465,16.666667,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.500000,33.000000,1.666667,36.233778,10.000000,52.576827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665318,93,47,33.0,1,3384,2333.0,-1.0,-1.0,16,2691,8.0,1831.0,0.500000,0.680416,1465,31.250000,0.544407,1448,31.250000,0.53809,1164,31.250000,0.432553,473,12.500000,0.175771,333,12.500000,0.123746,40.870133,45.773048,37.750000,45.500000,25.833333,36.233778,51.666667,52.576827
87801,94,53,42.0,1,3384,2333.0,-1.0,-1.0,38,2691,30.0,1831.0,0.789474,0.680416,1465,78.947368,0.544407,1448,78.947368,0.53809,1164,78.947368,0.432553,473,78.947368,0.175771,333,71.052632,0.123746,40.870133,45.773048,53.589744,59.677419,78.632479,36.233778,98.924731,52.576827
886325,95,39,24.0,1,3384,2333.0,-1.0,-1.0,31,2691,20.0,1831.0,0.645161,0.680416,1465,51.612903,0.544407,1448,51.612903,0.53809,1164,51.612903,0.432553,473,6.451613,0.175771,333,6.451613,0.123746,40.870133,45.773048,38.990625,44.385000,29.968750,36.233778,47.950000,52.576827
163916,201,10,2.0,1,3384,2333.0,-1.0,-1.0,1,2691,0.0,1831.0,0.000000,0.680416,1465,0.000000,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.000000,NaN,0.000000,36.233778,NaN,52.576827


In [50]:
# Séparer les lignes à dupliquer
df_to_duplicate = df[df['dep'] == 75].copy()
df_to_duplicate['dup'] = 1  # marquer les duplications
# Marquer les lignes originales
df['dup'] = 0
# Fusionner les deux
df = pd.concat([df, df_to_duplicate], ignore_index=True)
df



,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,24,22.0,1,3384,2333.0,-1.0,-1.0,17,2691,17.0,1831.0,1.000000,0.680416,1465,94.117647,0.544407,1448,94.117647,0.53809,1164,94.117647,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,48.823529,48.823529,62.745098,36.233778,62.745098,52.576827,0
1,2,13,7.0,1,3384,2333.0,1.0,1.0,12,2691,6.0,1831.0,0.500000,0.680416,1465,25.000000,0.544407,1448,25.000000,0.53809,1164,25.000000,0.432553,473,8.333333,0.175771,333,8.333333,0.123746,40.870133,45.773048,35.769231,40.714286,19.230769,36.233778,35.714286,52.576827,0
2,3,19,11.0,1,3384,2333.0,1.0,1.0,19,2691,11.0,1831.0,0.578947,0.680416,1465,47.368421,0.544407,1448,47.368421,0.53809,1164,36.842105,0.432553,473,26.315789,0.175771,333,26.315789,0.123746,40.870133,45.773048,40.526316,48.181818,35.087719,36.233778,60.606061,52.576827,0
3,4,3,2.0,1,3384,2333.0,-1.0,-1.0,2,2691,1.0,1831.0,0.500000,0.680416,1465,50.000000,0.544407,1448,50.000000,0.53809,1164,50.000000,0.432553,473,50.000000,0.175771,333,50.000000,0.123746,40.870133,45.773048,45.000000,60.000000,50.000000,36.233778,100.000000,52.576827,0
4,5,8,3.0,1,3384,2333.0,-1.0,-1.0,6,2691,1.0,1831.0,0.166667,0.680416,1465,16.666667,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.500000,33.000000,1.666667,36.233778,10.000000,52.576827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,94,53,42.0,1,3384,2333.0,-1.0,-1.0,38,2691,30.0,1831.0,0.789474,0.680416,1465,78.947368,0.544407,1448,78.947368,0.53809,1164,78.947368,0.432553,473,78.947368,0.175771,333,71.052632,0.123746,40.870133,45.773048,53.589744,59.677419,78.632479,36.233778,98.924731,52.576827,0
88,95,39,24.0,1,3384,2333.0,-1.0,-1.0,31,2691,20.0,1831.0,0.645161,0.680416,1465,51.612903,0.544407,1448,51.612903,0.53809,1164,51.612903,0.432553,473,6.451613,0.175771,333,6.451613,0.123746,40.870133,45.773048,38.990625,44.385000,29.968750,36.233778,47.950000,52.576827,0
89,201,10,2.0,1,3384,2333.0,-1.0,-1.0,1,2691,0.0,1831.0,0.000000,0.680416,1465,0.000000,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.000000,NaN,0.000000,36.233778,NaN,52.576827,0
90,202,7,0.0,1,3384,2333.0,-1.0,-1.0,2,2691,0.0,1831.0,0.000000,0.680416,1465,0.000000,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.000000,NaN,0.000000,36.233778,NaN,52.576827,0


In [51]:
df.loc[df["dup"] == 1, "dep"] = 0
df.loc[df["dup"] == 1, "NB_total"] = df.loc[df["dup"] == 1]["NB_Ftotal"]
df.loc[df["dup"] == 1, "NB2_total"] = df.loc[df["dup"] == 1]["NB2_Ftotal"]
df.loc[df["dup"] == 1, "NB"] = df.loc[df["dup"] == 1]["NB_F"]
df.loc[df["dup"] == 1, "NB2"] = df.loc[df["dup"] == 1]["NB2_F"]
df.loc[df["dup"] == 1, "ShareS2"] = df.loc[df["dup"] == 1]["ShareS2_F"]
df.loc[df["dup"] == 1, "r10"] = df.loc[df["dup"] == 1]["r10_F"]
df.loc[df["dup"] == 1, "r25"] = df.loc[df["dup"] == 1]["r25_F"]
df.loc[df["dup"] == 1, "r50"] = df.loc[df["dup"] == 1]["r50_F"]
df.loc[df["dup"] == 1, "r100"] = df.loc[df["dup"] == 1]["r100_F"]
df.loc[df["dup"] == 1, "PrixMoyen"] = df.loc[df["dup"] == 1]["PF"]
df.loc[df["dup"] == 1, "PrixMoyenS2"] = df.loc[df["dup"] == 1]["PFS2"]
df.loc[df["dup"] == 1, "depmoyen"] = df.loc[df["dup"] == 1]["depmoyen_F"]
df.loc[df["dup"] == 1, "depmoyenS2"] = df.loc[df["dup"] == 1]["depmoyen_FS2"]
df

,dep,NB_total,NB2_total,c,NB_Ftotal,NB2_Ftotal,cc,ccc,NB,NB_F,NB2,NB2_F,ShareS2,ShareS2_F,c10_F,r10,r10_F,c25_F,r25,r25_F,c50_F,r50,r50_F,c75_F,r75,r75_F,c100_F,r100,r100_F,PF,PFS2,PrixMoyen,PrixMoyenS2,depmoyen,depmoyen_F,depmoyenS2,depmoyen_FS2,dup
0,1,24,22.0,1,3384,2333.0,-1.0,-1.0,17,2691,17.0,1831.0,1.000000,0.680416,1465,94.117647,0.544407,1448,94.117647,0.53809,1164,94.117647,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,48.823529,48.823529,62.745098,36.233778,62.745098,52.576827,0
1,2,13,7.0,1,3384,2333.0,1.0,1.0,12,2691,6.0,1831.0,0.500000,0.680416,1465,25.000000,0.544407,1448,25.000000,0.53809,1164,25.000000,0.432553,473,8.333333,0.175771,333,8.333333,0.123746,40.870133,45.773048,35.769231,40.714286,19.230769,36.233778,35.714286,52.576827,0
2,3,19,11.0,1,3384,2333.0,1.0,1.0,19,2691,11.0,1831.0,0.578947,0.680416,1465,47.368421,0.544407,1448,47.368421,0.53809,1164,36.842105,0.432553,473,26.315789,0.175771,333,26.315789,0.123746,40.870133,45.773048,40.526316,48.181818,35.087719,36.233778,60.606061,52.576827,0
3,4,3,2.0,1,3384,2333.0,-1.0,-1.0,2,2691,1.0,1831.0,0.500000,0.680416,1465,50.000000,0.544407,1448,50.000000,0.53809,1164,50.000000,0.432553,473,50.000000,0.175771,333,50.000000,0.123746,40.870133,45.773048,45.000000,60.000000,50.000000,36.233778,100.000000,52.576827,0
4,5,8,3.0,1,3384,2333.0,-1.0,-1.0,6,2691,1.0,1831.0,0.166667,0.680416,1465,16.666667,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.500000,33.000000,1.666667,36.233778,10.000000,52.576827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,94,53,42.0,1,3384,2333.0,-1.0,-1.0,38,2691,30.0,1831.0,0.789474,0.680416,1465,78.947368,0.544407,1448,78.947368,0.53809,1164,78.947368,0.432553,473,78.947368,0.175771,333,71.052632,0.123746,40.870133,45.773048,53.589744,59.677419,78.632479,36.233778,98.924731,52.576827,0
88,95,39,24.0,1,3384,2333.0,-1.0,-1.0,31,2691,20.0,1831.0,0.645161,0.680416,1465,51.612903,0.544407,1448,51.612903,0.53809,1164,51.612903,0.432553,473,6.451613,0.175771,333,6.451613,0.123746,40.870133,45.773048,38.990625,44.385000,29.968750,36.233778,47.950000,52.576827,0
89,201,10,2.0,1,3384,2333.0,-1.0,-1.0,1,2691,0.0,1831.0,0.000000,0.680416,1465,0.000000,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.000000,NaN,0.000000,36.233778,NaN,52.576827,0
90,202,7,0.0,1,3384,2333.0,-1.0,-1.0,2,2691,0.0,1831.0,0.000000,0.680416,1465,0.000000,0.544407,1448,0.000000,0.53809,1164,0.000000,0.432553,473,0.000000,0.175771,333,0.000000,0.123746,40.870133,45.773048,30.000000,NaN,0.000000,36.233778,NaN,52.576827,0


In [52]:
# NB_Ftotal  NB2_Ftotal NB_F NB2_F ShareS2_F r10_F r25_F r50_F r100_F PF PFS2 depmoyen_F depmoyen_FS2 c10_F c25_F c50_F c100_F dup c
df = df.drop(["NB_Ftotal","NB2_Ftotal","NB_F","NB2_F","ShareS2_F","r10_F","r25_F","r50_F","r100_F","PF","PFS2","depmoyen_F","depmoyen_FS2","c10_F","c25_F","c50_F","c75_F","r75_F", "r75","c100_F","dup","c","cc","ccc"], axis=1)
df

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,24,22.0,17,17.0,1.000000,94.117647,94.117647,94.117647,0.000000,48.823529,48.823529,62.745098,62.745098
1,2,13,7.0,12,6.0,0.500000,25.000000,25.000000,25.000000,8.333333,35.769231,40.714286,19.230769,35.714286
2,3,19,11.0,19,11.0,0.578947,47.368421,47.368421,36.842105,26.315789,40.526316,48.181818,35.087719,60.606061
3,4,3,2.0,2,1.0,0.500000,50.000000,50.000000,50.000000,50.000000,45.000000,60.000000,50.000000,100.000000
4,5,8,3.0,6,1.0,0.166667,16.666667,0.000000,0.000000,0.000000,30.500000,33.000000,1.666667,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,94,53,42.0,38,30.0,0.789474,78.947368,78.947368,78.947368,71.052632,53.589744,59.677419,78.632479,98.924731
88,95,39,24.0,31,20.0,0.645161,51.612903,51.612903,51.612903,6.451613,38.990625,44.385000,29.968750,47.950000
89,201,10,2.0,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,NaN,0.000000,NaN
90,202,7,0.0,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,NaN,0.000000,NaN


In [53]:
# df[df["dep"] == 38]
df = df.reset_index(drop=True)
df.head(10)


,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,PrixMoyenS2,depmoyen,depmoyenS2
0,1,24,22.0,17,17.0,1.000000,94.117647,94.117647,94.117647,0.000000,48.823529,48.823529,62.745098,62.745098
1,2,13,7.0,12,6.0,0.500000,25.000000,25.000000,25.000000,8.333333,35.769231,40.714286,19.230769,35.714286
2,3,19,11.0,19,11.0,0.578947,47.368421,47.368421,36.842105,26.315789,40.526316,48.181818,35.087719,60.606061
3,4,3,2.0,2,1.0,0.500000,50.000000,50.000000,50.000000,50.000000,45.000000,60.000000,50.000000,100.000000
4,5,8,3.0,6,1.0,0.166667,16.666667,0.000000,0.000000,0.000000,30.500000,33.000000,1.666667,10.000000
5,6,105,80.0,64,49.0,0.765625,65.625000,65.625000,65.625000,26.562500,47.587692,52.864000,58.625641,76.213333
6,7,20,10.0,19,9.0,0.473684,36.842105,36.842105,36.842105,0.000000,35.250000,41.666667,17.500000,38.888889
7,8,7,2.0,5,2.0,0.400000,0.000000,0.000000,0.000000,0.000000,30.000000,30.000000,0.000000,0.000000
8,10,11,5.0,10,5.0,0.500000,50.000000,50.000000,50.000000,50.000000,45.000000,60.000000,50.000000,100.000000
9,11,16,8.0,13,6.0,0.461538,23.076923,23.076923,0.000000,0.000000,32.307692,35.000000,7.692308,16.666667


In [54]:
df2 = pd.read_csv("../data/depassement/Rendus_PartIII/dépassements_anest.csv")
df2.head(10)

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,depmoyen,PrixMoyenS2,depmoyenS2
0,1,24,22,17,17,1.000000,94.117645,94.117645,94.117645,0.000000,48.823528,62.745,48.823528,62.745
1,2,13,7,12,6,0.500000,25.000000,25.000000,25.000000,8.333333,35.769230,19.231,40.714287,35.714
2,3,19,11,19,11,0.578947,47.368420,47.368420,36.842106,26.315790,40.526318,35.088,48.181820,60.606
3,4,3,2,2,1,0.500000,50.000000,50.000000,50.000000,50.000000,45.000000,50.000,60.000000,100.000
4,5,8,3,6,1,0.166667,16.666666,0.000000,0.000000,0.000000,30.500000,1.667,33.000000,10.000
5,6,105,80,64,49,0.765625,65.625000,65.625000,65.625000,26.562500,47.587692,58.626,52.863998,76.213
6,7,20,10,19,9,0.473684,36.842106,36.842106,36.842106,0.000000,35.250000,17.500,41.666668,38.889
7,8,7,2,5,2,0.400000,0.000000,0.000000,0.000000,0.000000,30.000000,0.000,30.000000,0.000
8,10,11,5,10,5,0.500000,50.000000,50.000000,50.000000,50.000000,45.000000,50.000,60.000000,100.000
9,11,16,8,13,6,0.461538,23.076923,23.076923,0.000000,0.000000,32.307693,7.692,35.000000,16.667


In [55]:
df1 = df.fillna(9999).round(2)
df2 = df2.fillna(9999).round(2)
diff = df1 - df2
pd.set_option('display.max_rows', 100)
diff.head(96)


,NB,NB2,NB2_total,NB_total,PrixMoyen,PrixMoyenS2,ShareS2,dep,depmoyen,depmoyenS2,r10,r100,r25,r50
0,0,0.0,0.0,0,0.0,0.00,0.0,0,0.01,0.01,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
5,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
6,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
7,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
8,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0
9,0,0.0,0.0,0,0.0,0.00,0.0,0,0.00,0.00,0.0,0.0,0.0,0.0


In [56]:
diff = df1["NB"].equals(df2["NB"])
diff = df1[df1["NB2"] != df2["NB2"]]
diff = df1[df1["NB2_total"] != df2["NB2_total"]]
diff = df1[df1["NB_total"] != df2["NB_total"]]
diff = df1[df1["PrixMoyen"] != df2["PrixMoyen"]]
diff = df2[df1["PrixMoyenS2"] != df2["PrixMoyenS2"]] # Aucune différence avec STATA, petite différence avec le fichier, + grosse différence avec SQL
diff

,dep,NB_total,NB2_total,NB,NB2,ShareS2,r10,r25,r50,r100,PrixMoyen,depmoyen,PrixMoyenS2,depmoyenS2
88,95,39,24,31,20,0.65,51.61,51.61,51.61,6.45,38.99,29.97,44.38,47.95


In [57]:
df.to_csv(f"../data/depassement/out/depassement_{profession_type}_{datesource_min}_{datesource_max}_{tarif_s1}_{study_id}.csv", index=False)

In [58]:
conn = psycopg2.connect(config.connection_string)
sql = f"delete from depassement where depassement_study_id={study_id}" # Ne marche pas la 1ère fois
with conn.cursor() as cur:
    cur.execute(sql)
conn.commit()
conn.close()
df["depassement_study_id"] = study_id
df.to_sql("depassement", config.connection_string, if_exists="append", index_label="id")
